In [ ]:
import networkx as nx
import dgl
import numpy as np

In [85]:
import pandas as pd
rating_df = pd.read_csv('./data/rating_data.csv', sep=',')
rating_df

,user,item,rating
0,1,104,2.0
1,1,106,5.0
2,1,107,5.0
3,1,108,5.0
4,1,109,3.0
...,...,...,...
273974,43486,19679,2.0
273975,43486,363,3.0
273976,43486,44038,2.0
273977,43486,61575,5.0


In [86]:
link_df = pd.read_csv('./data/link_data.csv', sep=',')
link_df

,user1,user2,weight
0,2824,2696,1.0
1,2824,18333,1.0
2,2824,10308,1.0
3,2824,1176,1.0
4,2824,208,1.0
...,...,...,...
303308,615,1294,1.0
303309,615,1096,1.0
303310,615,145,1.0
303311,615,1499,1.0


In [ ]:
rating_df.user.value_counts().sort_index().index

In [ ]:
link_df.user1.value_counts().sort_index().index

In [ ]:
# 保证rating和link里面的user一致
(~rating_df.user.value_counts().sort_index().index == link_df.user1.value_counts().sort_index().index).sum()

In [ ]:
(~rating_df.user.value_counts().sort_index().index == link_df.user2.value_counts().sort_index().index).sum()

In [105]:
item_N = len(rating_df.item.value_counts())
item_N

19212

In [106]:
user_N = len(rating_df.user.value_counts())
user_N

7968

In [ ]:
u_array = np.arange(user_N)
u_array

In [ ]:
i_array = np.arange(user_N, user_N + item_N)
i_array

In [ ]:
# 生成字典
sorted_user_idx_lst = sorted(rating_df.user.value_counts().index)
sorted_user_idx_lst

In [ ]:
user_raw_idx2new_idx = {}
user_new_idx2raw_idx = {}
for i in range(len(sorted_user_idx_lst)):
    raw_id = sorted_user_idx_lst[i]
    new_id = int(u_array[i])
    # print(type(raw_id), type(new_id))
    user_raw_idx2new_idx[raw_id] = new_id
    user_new_idx2raw_idx[new_id] = raw_id
    # print(user_raw_idx2new_idx)

In [102]:
user_raw_idx2new_idx

{1: 0,
 2: 1,
 4: 2,
 5: 3,
 7: 4,
 8: 5,
 9: 6,
 11: 7,
 12: 8,
 14: 9,
 16: 10,
 19: 11,
 20: 12,
 21: 13,
 22: 14,
 23: 15,
 24: 16,
 27: 17,
 28: 18,
 29: 19,
 30: 20,
 31: 21,
 32: 22,
 33: 23,
 34: 24,
 35: 25,
 38: 26,
 40: 27,
 41: 28,
 42: 29,
 45: 30,
 46: 31,
 47: 32,
 48: 33,
 50: 34,
 51: 35,
 53: 36,
 56: 37,
 57: 38,
 58: 39,
 59: 40,
 60: 41,
 61: 42,
 62: 43,
 65: 44,
 66: 45,
 67: 46,
 68: 47,
 69: 48,
 71: 49,
 72: 50,
 73: 51,
 74: 52,
 76: 53,
 77: 54,
 78: 55,
 79: 56,
 80: 57,
 81: 58,
 82: 59,
 85: 60,
 86: 61,
 87: 62,
 89: 63,
 90: 64,
 91: 65,
 92: 66,
 93: 67,
 94: 68,
 95: 69,
 96: 70,
 99: 71,
 102: 72,
 103: 73,
 105: 74,
 108: 75,
 109: 76,
 110: 77,
 111: 78,
 112: 79,
 114: 80,
 115: 81,
 116: 82,
 117: 83,
 118: 84,
 120: 85,
 121: 86,
 122: 87,
 123: 88,
 124: 89,
 125: 90,
 127: 91,
 128: 92,
 129: 93,
 130: 94,
 132: 95,
 133: 96,
 134: 97,
 135: 98,
 136: 99,
 137: 100,
 138: 101,
 139: 102,
 140: 103,
 142: 104,
 143: 105,
 144: 106,
 145: 107,
 

In [ ]:
user_new_idx2raw_idx

In [ ]:
import json
user_idx_dict = {
    'raw2new': user_raw_idx2new_idx,
    'new2raw': user_new_idx2raw_idx
}
with open('./data/user_idx_transfer.json', 'w') as f:
    json.dump(user_idx_dict, f, indent=2)

In [ ]:
# 生成字典
sorted_item_idx_lst = sorted(rating_df.item.value_counts().index)
sorted_item_idx_lst

In [ ]:
item_raw_idx2new_idx = {}
item_new_idx2raw_idx = {}
for i in range(len(sorted_item_idx_lst)):
    raw_id = sorted_item_idx_lst[i]
    new_id = int(i_array[i])
    item_raw_idx2new_idx[raw_id] = new_id
    item_new_idx2raw_idx[new_id] = raw_id

In [ ]:
item_raw_idx2new_idx

In [ ]:
item_new_idx2raw_idx

In [ ]:
item_idx_dict = {
    'raw2new': item_raw_idx2new_idx,
    'new2raw': item_new_idx2raw_idx
}
with open('./data/item_idx_transfer.json', 'w') as f:
    json.dump(item_idx_dict, f, indent=2)

# 将原始数据的idx替换

In [100]:
new_rating_df = rating_df.copy()

In [101]:
new_rating_df

,user,item,rating
0,1,104,2.0
1,1,106,5.0
2,1,107,5.0
3,1,108,5.0
4,1,109,3.0
...,...,...,...
273974,43486,19679,2.0
273975,43486,363,3.0
273976,43486,44038,2.0
273977,43486,61575,5.0


In [103]:
new_rating_df['user'] = new_rating_df['user'].apply(lambda x: user_raw_idx2new_idx[x])
new_rating_df

,user,item,rating
0,0,104,2.0
1,0,106,5.0
2,0,107,5.0
3,0,108,5.0
4,0,109,3.0
...,...,...,...
273974,7967,19679,2.0
273975,7967,363,3.0
273976,7967,44038,2.0
273977,7967,61575,5.0


In [104]:
new_rating_df['item'] = new_rating_df['item'].apply(lambda x: item_raw_idx2new_idx[x])
new_rating_df

,user,item,rating
0,0,7985,2.0
1,0,7986,5.0
2,0,7987,5.0
3,0,7988,5.0
4,0,7989,3.0
...,...,...,...
273974,7967,15068,2.0
273975,7967,8072,3.0
273976,7967,20443,2.0
273977,7967,23673,5.0


In [107]:
new_rating_df.to_csv('./data/rating_data.csv', index=False, header=True)

In [109]:
new_link_df = link_df.copy()
new_link_df

,user1,user2,weight
0,2824,2696,1.0
1,2824,18333,1.0
2,2824,10308,1.0
3,2824,1176,1.0
4,2824,208,1.0
...,...,...,...
303308,615,1294,1.0
303309,615,1096,1.0
303310,615,145,1.0
303311,615,1499,1.0


In [110]:
new_link_df['user1'] = new_link_df['user1'].apply(lambda x: user_raw_idx2new_idx[x])
new_link_df

,user1,user2,weight
0,2178,2696,1.0
1,2178,18333,1.0
2,2178,10308,1.0
3,2178,1176,1.0
4,2178,208,1.0
...,...,...,...
303308,497,1294,1.0
303309,497,1096,1.0
303310,497,145,1.0
303311,497,1499,1.0


In [111]:
new_link_df['user2'] = new_link_df['user2'].apply(lambda x: user_raw_idx2new_idx[x])
new_link_df

,user1,user2,weight
0,2178,2094,1.0
1,2178,6986,1.0
2,2178,5403,1.0
3,2178,998,1.0
4,2178,160,1.0
...,...,...,...
303308,497,1088,1.0
303309,497,928,1.0
303310,497,107,1.0
303311,497,1221,1.0


In [114]:
new_link_df.to_csv('./data/link_data.csv', index=False, header=True)